In [6]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [7]:
# ensemble csv files
submission_files = ['./work_dirs/Yolo_trash/submission_latest.csv',                  # 변경
                   './work_dirs/faster_rcnn_r50_fpn_1x_trash/submission_latest.csv',
                   './work_dirs/ensemble/faster_yolo.csv'] # 변경
submission_df = [pd.read_csv(file) for file in submission_files]

In [8]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []

annotation = '../../dataset/test.json'
coco = COCO(annotation)
image_ids = submission_df[0]['image_id'].tolist()


prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.4

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates < 0
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 9 boxes coordinates < 0
Warning. Fixed 16 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 7 boxes coordinates < 0
Warning. Fixed 9 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 18 boxes coordinates < 0
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 7 boxes coordinates < 0
Warning. Fixed 5 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 12 boxes coordinates < 0
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 10 boxes coordinates < 0
Warning. Fixed 7 boxes coordina

In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./work_dirs/ensemble/faster_yolo.csv')   # 변경

submission.head()

,PredictionString,image_id
0,0 0.5516212 481.6552 579.4817 558.7082 661.014...,test/0000.jpg
1,0 0.6251539 747.6425 647.7722 887.6109 973.435...,test/0001.jpg
2,0 0.8162597 880.744 474.80396 1017.0764 636.69...,test/0002.jpg
3,0 0.26036945 110.72623 89.16498 987.23413 961....,test/0003.jpg
4,0 0.8324693 411.59436 416.68414 669.80334 570....,test/0004.jpg
